# CIMD in OAuth — Minimal Flow Simulation

This notebook extends the minimal OAuth simulation
by introducing Client-ID Metadata Documents (CIMD).

Goal:
- Replace static client_id with URL-based identity
- Enforce CIMD invariants mechanically
- Keep the OAuth flow unchanged


## What CIMD Changes (and What It Does Not)

Changes:
- client_id is now a URL
- Authorization Server must fetch metadata
- Client identity becomes dynamic

Unchanged:
- Authorization Code flow
- PKCE
- Redirect URI matching
- Token issuance


In [1]:
# Simulated remote CIMD metadata documents
REMOTE_METADATA = {
    "https://agent.example.com/identity.json": {
        "client_id": "https://agent.example.com/identity.json",
        "redirect_uris": ["https://agent.example.com/callback"],
        "grant_types": ["authorization_code"],
        "scope": "research.read"
    }
}
